In [1]:
import re
import json
from lxml import html
import urllib.request

In [2]:
URL_BASE = 'http://www.mathnet.ru'

URL = '''http://www.mathnet.ru/php/archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017&volume=158&issue=1&option_lang=rus&bookID=1621'''

In [3]:
ANNOTATION_XPATH_EXPR = '''
    //b[contains(text(), 'Аннотация')]/following-sibling::text()[not(preceding-sibling::b[contains(text(), 'Ключевые')])]
'''

TITLE_XPATH_EXPR = '''
    //td//td//td//a[@class='SLink']
'''

KEYWORDS_XPATH_EXPR = '''
    //b[contains(text(), 'Ключевые')]/following-sibling::i
'''

In [4]:
class Stemmer:
    # Helper regex strings.
    _vowel = "[аеиоуыэюя]"
    _non_vowel = "[^аеиоуыэюя]"

    # Word regions.
    _re_rv = re.compile(_vowel)
    _re_r1 = re.compile(_vowel + _non_vowel)

    # Endings.
    _re_perfective_gerund = re.compile(
        r"(((?P<ignore>[ая])(в|вши|вшись))|(ив|ивши|ившись|ыв|ывши|ывшись))$"
    )
    _re_adjective = re.compile(
        r"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|"
        r"ую|юю|ая|яя|ою|ею)$"
    )
    _re_participle = re.compile(
        r"(((?P<ignore>[ая])(ем|нн|вш|ющ|щ))|(ивш|ывш|ующ))$"
    )
    _re_reflexive = re.compile(
        r"(ся|сь)$"
    )
    _re_verb = re.compile(
        r"(((?P<ignore>[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|"
        r"нно))|(ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|"
        r"ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю))$"
    )
    _re_noun = re.compile(
        r"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|"
        r"ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$"
    )
    _re_superlative = re.compile(
        r"(ейш|ейше)$"
    )
    _re_derivational = re.compile(
        r"(ост|ость)$"
    )
    _re_i = re.compile(
        r"и$"
    )
    _re_nn = re.compile(
        r"((?<=н)н)$"
    )
    _re_ = re.compile(
        r"ь$"
    )

    def stem(self, word):
        """
        Gets the stem.
        """

        rv_pos, r2_pos = self._find_rv(word), self._find_r2(word)
        word = self._step_1(word, rv_pos)
        word = self._step_2(word, rv_pos)
        word = self._step_3(word, r2_pos)
        word = self._step_4(word, rv_pos)
        return word

    def _find_rv(self, word):
        """
        Searches for the RV region.
        """

        rv_match = self._re_rv.search(word)
        if not rv_match:
            return len(word)
        return rv_match.end()

    def _find_r2(self, word):
        """
        Searches for the R2 region.
        """

        r1_match = self._re_r1.search(word)
        if not r1_match:
            return len(word)
        r2_match = self._re_r1.search(word, r1_match.end())
        if not r2_match:
            return len(word)
        return r2_match.end()

    def _cut(self, word, ending, pos):
        """
        Tries to cut the specified ending after the specified position.
        """

        match = ending.search(word, pos)
        if match:
            try:
                ignore = match.group("ignore") or ""
            except IndexError:
                # No ignored characters in pattern.
                return True, word[:match.start()]
            else:
                # Do not cut ignored part.
                return True, word[:match.start() + len(ignore)]
        else:
            return False, word

    def _step_1(self, word, rv_pos):
        match, word = self._cut(word, self._re_perfective_gerund, rv_pos)
        if match:
            return word
        _, word = self._cut(word, self._re_reflexive, rv_pos)
        match, word = self._cut(word, self._re_adjective, rv_pos)
        if match:
            _, word = self._cut(word, self._re_participle, rv_pos)
            return word
        match, word = self._cut(word, self._re_verb, rv_pos)
        if match:
            return word
        _, word = self._cut(word, self._re_noun, rv_pos)
        return word

    def _step_2(self, word, rv_pos):
        _, word = self._cut(word, self._re_i, rv_pos)
        return word

    def _step_3(self, word, r2_pos):
        _, word = self._cut(word, self._re_derivational, r2_pos)
        return word

    def _step_4(self, word, rv_pos):
        _, word = self._cut(word, self._re_superlative, rv_pos)
        match, word = self._cut(word, self._re_nn, rv_pos)
        if not match:
            _, word = self._cut(word, self._re_, rv_pos)
        return word

In [5]:
stemmer = Stemmer()

In [8]:
from pymystem3 import Mystem

mystem = Mystem()

Installing mystem to /Users/alex/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-macosx10.8.tar.gz


In [6]:
def main():
    main_page = html.fromstring(urllib.request.urlopen(URL).read())

    articles = []
    
    links = list(filter(lambda x: x.text is not None, main_page.xpath(TITLE_XPATH_EXPR)))
    for link in links:
        article_page = html.fromstring(urllib.request.urlopen(URL_BASE + link.get("href")).read())

        title = link.text;
        title_porter = ' '.join(map(stemmer.stem, title.split(' ')))
        title_mystem = 
        
        annotation = str.join('', article_page.xpath(ANNOTATION_XPATH_EXPR))
        annotation = re.sub('\t|\n', '', annotation)
        annotation_porter = ' '.join(map(stemmer.stem, annotation.split(' ')))
        annotation_mystem = ''.join(mystem.lemmatize(annotation))
        
        keywords = list(filter(None, re.split(', ', article_page.xpath(KEYWORDS_XPATH_EXPR)[0].text)))

        articles.append({
            'title': {
                'origin': title,
                'porter': title_porter,
                'mystem': title_mystem
            },
            'link': URL_BASE + link.get("href"),
            'annotation': {
                'origin': annotation,
                'porter': annotation_porter,
                'mystem': annotation_mystem
            },
            'keywords': keywords
        })

    result = {
        'issue': {
            'URL': URL,
            'articles': articles
        }
    }
        
    print(json.dumps(result, ensure_ascii=False, indent=4, sort_keys=True))

In [7]:
main()

{
    "issue": {
        "URL": "http://www.mathnet.ru/php/archive.phtml?jrnid=uzku&wshow=issue&bshow=contents&series=0&year=2017&volume=158&issue=1&option_lang=rus&bookID=1621",
        "articles": [
            {
                "annotation": {
                    "origin": "В работе рассмотрен класс многочленов типа Капелли в свободной ассоциативной алгебре , где  – произвольное поле,  – счетное множество, обобщающий конструкцию кратных многочленов Капелли. Приведены основные свойства введенных многочленов. В частности, указано их разложение через многочлены того же вида и установлены некоторые соотношения между их -идеалами. Кроме того, установлена связь между двойными многочленами Капелли и квазимногочленами Капелли.",
                    "porter": "В работ рассмотр класс многочлен тип Капелл в свободн ассоциативн алгебр , где  – произвольн поле,  – счетн множество, обобща конструкц кратн многочлен Капелли. Привед основн свойств введен многочленов. В частности, указа их разложен ч